In [95]:
import pandas as pd 
import hashlib
import os 
from utils import logger
def file_as_bytes(file):
    with file:
        return file.read()

In [96]:
import numpy as np
pd.set_option('display.max_columns', None)
df = pd.read_csv("/Users/xin/Downloads/EE542/Labs/lab10/data/files_meta.tsv", sep="\t")
df_case = pd.read_csv("/Users/xin/Downloads/EE542/Labs/lab10/data/cases_meta.tsv", sep="\t")
df_map = pd.read_csv("/Users/xin/Downloads/EE542/Labs/lab10/data/file_case_id_DNA.csv")

/Users/xin/anaconda/envs/py3k/lib/python3.7/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (1,5,9,10,11,12,13,14,15,16,17,18,21,22,23,24,25,26,29,30,31,32,33,34,35,36,37,38,39,40,45,50,54,55,56,57,58,59,71,72,73,74,75,76,77,78,79,80,81,84,86,87,89,90,92,97,98,99,110,111,112,113,114,115,117,124,125,136,138,139,140,141,142,143,144,145,146,148,149,151,152,165,177,178,179,180,181,182,183,184,185,186,187,191,192,194,200,203,204,205,214,215,217,218,219,222,223,224,225,226,227,228,229,230,231,233,234,242,243,254,264,265,266,267,268,269,270,271,273,274,275,276,277,278,279,280,282,283,284,285,286,287,288,289,290) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [97]:
df.loc[df['cases.0.samples.0.sample_type'].str.contains("Normal"), 'label'] = 0
df.loc[~df['cases.0.samples.0.sample_type'].str.contains("Normal"), 'label'] = 1

In [98]:
for i in range(450,460):#,df.shape[0]):#rows
    print(df.loc[i,'file_id'])
    cur = df_map.loc[df_map['file_id']==df.loc[i,'file_id']]
    case_id = cur['case_id'].item()
    print(case_id)
    cur_case = df_case.loc[df_case['case_id']==case_id]
    cur_label = cur_case.primary_site.item()
    print(cur_label)
    df.loc[i,'label'] = cur_label
    print(df.loc[i, 'label'])

5e188175-1acc-4492-af53-8eef15400b61
2038fd65-d8f1-4b16-af90-b1c8f9a379a7
Ovary
Ovary
b10221df-896b-449f-8526-a606763c95f0
c5355491-e1e8-46a4-a05e-bafcaf2e7459
Ovary
Ovary
fe884482-47e2-4ea4-9351-d464fd28c06b
446ce2a3-d328-443c-a419-3344baad0e16
Ovary
Ovary
bdc72569-8da5-45fd-9633-17c38481a370
78777a80-cc6c-46c9-a14a-837ac7943719
Ovary
Ovary
586bcf84-4b83-458f-8d9d-1cdc39b4af83
d7925dfc-18ce-46ab-a47b-9d06eacc96d0
Ovary
Ovary
befb4951-1e89-4d8b-beab-466200e86caf
e641aed9-1dd8-4c30-b231-f12b20a76df0
Ovary
Ovary
1fd0d5bb-732b-4537-b456-ecdb44ed4fbe
94efd4f9-69b0-4efa-8a0c-61106e898fdd
Ovary
Ovary
51d8da81-e323-4f90-8b46-8da93d289ee4
56a30462-2819-4c18-95be-8e73880a4921
Ovary
Ovary
a76d1f5a-76dc-46de-9f27-8006dc0c9c64
1e14f098-17b2-4f54-8e4f-e38088008df7
Ovary
Ovary
b5109358-bfe3-4fe3-93be-b123fa43a85e
2a924999-1fbf-470a-97bd-fd2276a9a366
Ovary
Ovary


In [99]:
    df_case.loc[df_case['file_id'] == df_map].md5.values[0] != filehash:
    
df['primary_site'] = df_case['primary_site']
df.label=np.where(df.label==1, df.primary_site, df.label) 
tumor_count = df.loc[df.label != 1].shape[0]
normal_count = df.loc[df.label == 0].shape[0]
logger.info("{} Normal samples, {} Tumor samples ".format(normal_count,tumor_count))

SyntaxError: invalid syntax (<ipython-input-99-8a2437cbcfce>, line 1)

In [ ]:
import numpy as np
df['primary_site'] = df_case['primary_site']
print (df['primary_site'].shape)
#df.head()
df.label=np.where(df.label==1,df.primary_site,df.label) 
#df.groupby('primary_site').nunique()
df_case.primary_site.unique()
len(df_case.primary_site.unique())
#df['label'].apply(lambda x:int(x))

In [ ]:
def extractMatrix(dirname):
	'''
	return a dataframe of the miRNA matrix, each row is the miRNA counts for a file_id

	'''
	count = 0

	miRNA_data = []
	for idname in os.listdir(dirname):
		# list all the ids 
		if idname.find("-") != -1:
			idpath = dirname +"/" + idname

			# all the files in each id directory
			for filename in os.listdir(idpath):
				# check the miRNA file
				if filename.find("-") != -1:

					filepath = idpath + "/" + filename
					df = pd.read_csv(filepath,sep="\t")
					# columns = ["miRNA_ID", "read_count"]
					if count ==0:
						# get the miRNA_IDs 
						miRNA_IDs = df.miRNA_ID.values.tolist()

					id_miRNA_read_counts = [idname] + df.read_count.values.tolist()
					miRNA_data.append(id_miRNA_read_counts)


					count +=1
					# print (df)
	columns = ["file_id"] + miRNA_IDs
	df = pd.DataFrame(miRNA_data, columns=columns)
	return df

In [ ]:
def extractLabel(inputfile, casefile):
    df = pd.read_csv(inputfile, sep="\t")
    df_case = pd.read_csv(casefile, sep="\t")
    #
    # print (df[columns])
    df['label'] = df['cases.0.samples.0.sample_type']
    df['primary_site'] = df_case['primary_site']
    print (df['primary_site'].shape)
    """
	df.loc[df['cases.0.samples.0.sample_type'].str.contains("Normal"), 'label'] = 0
	df.loc[df['cases.0.samples.0.sample_type'].str.contains("Tumor"), 'label'] = 1
	tumor_count = df.loc[df.label == 1].shape[0]
	normal_count = df.loc[df.label == 0].shape[0]
	logger.info("{} Normal samples, {} Tumor samples ".format(normal_count,tumor_count))
	columns = ['file_id','label']
    """
    df.loc[df['cases.0.samples.0.sample_type'].str.contains("Normal"), 'label'] = 0
    df.loc[df['cases.0.samples.0.sample_type'].str.contains("Tumor"), 'label'] = 1
    df.head()
    tumor_count = df.loc[df.label == 1].shape[0]
    normal_count = df.loc[df.label == 0].shape[0]
    logger.info("{} Normal samples, {} Tumor samples ".format(normal_count,tumor_count))
    columns = ['file_id','label']
    return df[columns]

In [ ]:
data_dir ="/Users/xin/Downloads/EE542/Labs/lab10/data/"
# Input directory and label file. The directory that holds the data. Modify this when use.
dirname = data_dir + "miRNA"
label_file = data_dir + "files_meta.tsv"
case_file = data_dir + "cases_meta.tsv"
#output file
outputfile = data_dir + "miRNA_matrix.csv"

# extract data
matrix_df = extractMatrix(dirname)
label_df = extractLabel(label_file, case_file)

In [ ]:
import pandas as pd 
import hashlib
import os 
from utils import logger
def file_as_bytes(file):
    with file:
        return file.read()

def extractMatrix(dirname):
	'''
	return a dataframe of the miRNA matrix, each row is the miRNA counts for a file_id

	'''
	count = 0

	miRNA_data = []
	for idname in os.listdir(dirname):
		# list all the ids 
		if idname.find("-") != -1:
			idpath = dirname +"/" + idname

			# all the files in each id directory
			for filename in os.listdir(idpath):
				# check the miRNA file
				if filename.find("-") != -1:

					filepath = idpath + "/" + filename
					df = pd.read_csv(filepath,sep="\t")
					# columns = ["miRNA_ID", "read_count"]
					if count ==0:
						# get the miRNA_IDs 
						miRNA_IDs = df.miRNA_ID.values.tolist()

					id_miRNA_read_counts = [idname] + df.read_count.values.tolist()
					miRNA_data.append(id_miRNA_read_counts)


					count +=1
					# print (df)
	columns = ["file_id"] + miRNA_IDs
	df = pd.DataFrame(miRNA_data, columns=columns)
	return df

def extractLabel(inputfile, casefile):
	df = pd.read_csv(inputfile, sep="\t")
	df_case = pd.read_csv(casefile, sep="\t")
	#
	# print (df[columns])
	df['label'] = df['cases.0.samples.0.sample_type']
	df['primary_site'] = df_case['primary_site']
	print df['primary_site'].shape
	"""
	df.loc[df['cases.0.samples.0.sample_type'].str.contains("Normal"), 'label'] = 0
	df.loc[df['cases.0.samples.0.sample_type'].str.contains("Tumor"), 'label'] = 1
	tumor_count = df.loc[df.label == 1].shape[0]
	normal_count = df.loc[df.label == 0].shape[0]
	logger.info("{} Normal samples, {} Tumor samples ".format(normal_count,tumor_count))
	columns = ['file_id','label']
	"""
	df.loc[df['cases.0.samples.0.sample_type'].str.contains("Normal"), 'label'] = 0
	df.loc[df['cases.0.samples.0.sample_type'].str.contains("Tumor"), 'label'] = 1

	tumor_count = df.loc[df.label == 1].shape[0]
	normal_count = df.loc[df.label == 0].shape[0]
	logger.info("{} Normal samples, {} Tumor samples ".format(normal_count,tumor_count))
	columns = ['file_id','label']
	return df[columns]

if __name__ == '__main__':


	data_dir ="/Users/xin/Downloads/EE542/Labs/lab10/data/"
	# Input directory and label file. The directory that holds the data. Modify this when use.
	dirname = data_dir + "miRNA"
	label_file = data_dir + "files_meta.tsv"
	case_file = data_dir + "cases_meta.tsv"
	#output file
	outputfile = data_dir + "miRNA_matrix.csv"

	# extract data
	matrix_df = extractMatrix(dirname)
	label_df = extractLabel(label_file, case_file)

	#merge the two based on the file_id
	result = pd.merge(matrix_df, label_df, on='file_id', how="left")
	#print(result)

	#save data
	result.to_csv(outputfile, index=False)
	#print (labeldf)
 